## MSE variables

mse:

$h = c_{p}T + gz + L_{v}q$

<br />
development of spatial variance of mass weighted vertically integrated mse:

$\frac{1}{2} \hat{\frac{\delta {h ^ \prime} ^2}{\delta t}} = \text{forcing correlation terms}$

$h ^ \prime$ - moist static energy anomaly from the domain mean \
$\delta t$ - one day


&#8291;1.  ta    –  temperature air \
2. gz    –  geopotential height \
3. hus   -  humidity specific

<br />

In [13]:
import intake
import xarray as xr
import xesmf as xe

import numpy as np
import skimage.measure as skm
import scipy

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat

import os
import myFuncs
import myPlots

In [14]:
model='MPI-ESM1-2-HR'
experiment_id='historical'
period = slice('1970-01-01','1999-12-31')
member_id='r1i1p1f1'


## ta   -  temperature air

In [15]:
table_id='day'
variable_id = 'ta'
ds_dict= intake.cat.nci['esgf'].cmip6.search(
                                source_id=model, 
                                experiment_id=experiment_id, 
                                member_id=member_id, 
                                variable_id=variable_id, 
                                table_id=table_id).to_dataset_dict()
ds = ds_dict[list(ds_dict.keys())[-1]].sel(time=period, lon=slice(0,360),lat=slice(-30,30))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version'


RuntimeError: can't start new thread

In [ ]:
ds

<xarray.Dataset>
Dimensions:    (time: 10957, bnds: 2, plev: 8, lat: 64, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1826, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 -29.92 -28.99 -28.99 ... 28.99 28.99 29.92
    lon_bnds   (lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 358.6 359.5
    ta         (time, plev, lat, lon) float32 dask.array<chunksize=(1826, 8, 64, 384), meta=np.ndarray>
Attributes: (12/49)
    realm:                   atmos
    sub_experiment_id:       none
    data_specs_version:      01.00.30
    sub_experiment:          none
    Conventions:             CF-1.7 CMIP-6.2
    variable_id:             ta
    ...                      ...
    parent_source_id:        MPI-ESM1-2-HR
    experiment_id:           historical
    source:                  MPI-ESM1.2-HR (2017): \naerosol: none, prescribe...
    institution:             Max Planck Institute for Meteorology, Hamburg 20...
    license:                 CMIP6 model data produced by MPI-M is licensed u...
    intake_esm_dataset_key:  CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1...

In [ ]:
ta=ds.ta #-273.15
#ta.attrs['units']= 'C\xb0'
ta

<xarray.DataArray 'ta' (time: 10957, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(10957, 8, 64, 384), dtype=float32, chunksize=(1827, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  air_temperature
    long_name:      Air Temperature
    comment:        Air Temperature
    units:          K
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
ta_test=ta.isel(time=slice(0, 4)) 
ta_test

<xarray.DataArray 'ta' (time: 4, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(4, 8, 64, 384), dtype=float32, chunksize=(4, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1970-01-04T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  air_temperature
    long_name:      Air Temperature
    comment:        Air Temperature
    units:          K
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
#da = ta.isel(time=0).interpolate_na(dim="plev", method='polynomial', order=2, use_coordinate="plev", keep_attrs=True)
# ta_dayVint = xr.DataArray(
#     data=-scipy.integrate.simpson(ta, ta.plev.data, axis=1, even='last')/ta.plev.data[0],
#     dims=['time','lat', 'lon'],
#     coords={'time': ta.time.data, 'lat': ta.lat.data, 'lon': ta.lon.data},
#     attrs={'units':'mm/day'}
#     )

In [ ]:
# myPlots.plot_snapshot(ta_dayVint, 'Reds', 'ta_Vint', model)

## zg    –  geopotential height

In [ ]:
table_id='day'
variable_id = 'zg'
ds_dict= intake.cat.nci['esgf'].cmip6.search(
                                source_id=model, 
                                experiment_id=experiment_id, 
                                member_id=member_id, 
                                variable_id=variable_id, 
                                table_id=table_id).to_dataset_dict()

ds = ds_dict[list(ds_dict.keys())[-1]].sel(time=period, lon=slice(0,360),lat=slice(-30,30))

#print(sorted(list(subset.unique()['table_id'].values())[1]))
#len(list(subset.unique()['table_id'].values())[1])


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version'


RuntimeError: can't start new thread

In [ ]:
ds

<xarray.Dataset>
Dimensions:    (time: 10957, bnds: 2, plev: 8, lat: 64, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1826, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 -29.92 -28.99 -28.99 ... 28.99 28.99 29.92
    lon_bnds   (lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 358.6 359.5
    zg         (time, plev, lat, lon) float32 dask.array<chunksize=(1826, 8, 64, 384), meta=np.ndarray>
Attributes: (12/49)
    branch_method:           standard
    creation_date:           2019-08-25T11:10:39Z
    mip_era:                 CMIP6
    parent_source_id:        MPI-ESM1-2-HR
    activity_id:             CMIP
    physics_index:           1
    ...                      ...
    table_info:              Creation Date:(09 May 2019) MD5:e6ef8ececc8f3386...
    branch_time_in_parent:   0.0
    initialization_index:    1
    cmor_version:            3.5.0
    parent_mip_era:          CMIP6
    intake_esm_dataset_key:  CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1...

In [ ]:
zg=ds.zg
zg

<xarray.DataArray 'zg' (time: 10957, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(10957, 8, 64, 384), dtype=float32, chunksize=(1827, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    comment:        Geopotential is the sum of the specific gravitational pot...
    units:          m
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
zg_test=zg.isel(time=slice(0, 4))
zg_test

<xarray.DataArray 'zg' (time: 4, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(4, 8, 64, 384), dtype=float32, chunksize=(4, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1970-01-04T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    comment:        Geopotential is the sum of the specific gravitational pot...
    units:          m
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
# da = zg.isel(time=0).interpolate_na(dim="plev", method='polynomial', order=2, use_coordinate="plev", keep_attrs=True)
# zg_dayVint = xr.DataArray(
#     data=-scipy.integrate.simpson(da, zg.plev.data, axis=1, even='last')/zg.plev.data[0],
#     dims=['time','lat', 'lon'],
#     coords={'time': zg.time.data, 'lat': zg.lat.data, 'lon': zg.lon.data},
#     attrs={'units':'mm/day'}
#     )

# zg_dayVint=zg.isel(time=0).mean(dim='plev', keep_attrs=True) #.sel(plev=850e2)

In [ ]:
# myPlots.plot_snapshot(zg_dayVint, 'Greys', 'zg_Vint', model)

## hus   -  humidity specific

In [ ]:
table_id='day'
variable_id = 'hus'
ds_dict= intake.cat.nci['esgf'].cmip6.search(
                                source_id=model, 
                                experiment_id=experiment_id, 
                                member_id=member_id, 
                                variable_id=variable_id, 
                                table_id=table_id).to_dataset_dict() 
                                
ds = ds_dict[list(ds_dict.keys())[-1]].sel(time=period, lon=slice(0,360),lat=slice(-30,30))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version'


In [ ]:
ds

<xarray.Dataset>
Dimensions:    (time: 10957, bnds: 2, plev: 8, lat: 64, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * lat        (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1826, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 -29.92 -28.99 -28.99 ... 28.99 28.99 29.92
    lon_bnds   (lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 358.6 359.5
    hus        (time, plev, lat, lon) float32 dask.array<chunksize=(1826, 8, 64, 384), meta=np.ndarray>
Attributes: (12/49)
    branch_method:           standard
    creation_date:           2019-08-25T11:10:40Z
    mip_era:                 CMIP6
    parent_source_id:        MPI-ESM1-2-HR
    activity_id:             CMIP
    physics_index:           1
    ...                      ...
    table_info:              Creation Date:(09 May 2019) MD5:e6ef8ececc8f3386...
    branch_time_in_parent:   0.0
    initialization_index:    1
    cmor_version:            3.5.0
    parent_mip_era:          CMIP6
    intake_esm_dataset_key:  CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1...

In [ ]:
hus=ds.hus #*1000
#hus.attrs['units']= 'g/kg'
hus

<xarray.DataArray 'hus' (time: 10957, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(10957, 8, 64, 384), dtype=float32, chunksize=(1827, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-31T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  specific_humidity
    long_name:      Specific Humidity
    comment:        Specific humidity is the mass fraction of water vapor in ...
    units:          1
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
hus_test=hus.isel(time=slice(0, 4))
hus_test

<xarray.DataArray 'hus' (time: 4, plev: 8, lat: 64, lon: 384)>
dask.array<getitem, shape=(4, 8, 64, 384), dtype=float32, chunksize=(4, 8, 64, 384), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1970-01-04T12:00:00
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Attributes:
    standard_name:  specific_humidity
    long_name:      Specific Humidity
    comment:        Specific humidity is the mass fraction of water vapor in ...
    units:          1
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-08-25T06:42:13Z altered by CMOR: Reordered dimension...

In [ ]:
# da = hus.fillna(0)
# hus_dayVint = xr.DataArray(
#     data=-scipy.integrate.simpson(da, hus.plev.data, axis=1, even='last')/hus.plev.data[0],
#     dims=['time','lat', 'lon'],
#     coords={'time': hus.time.data, 'lat': hus.lat.data, 'lon': hus.lon.data}
#     ,attrs={'units':'mm/day'}
#     )

In [ ]:
# myPlots.plot_snapshot(hus_dayVint, 'Greens', 'hus_Vint', model)

## mse

$h = c_{p}T + gz + L_{v}q$

In [ ]:
c_p = 1.005
L_v = 2.256e6
mse = c_p*ta + zg + L_v*hus
mse

NameError: name 'zg' is not defined

In [ ]:
mse_year = mse.sel(time = slice('1987-01-31','1987-12-31'))
xr.Dataset({'mse_year': mse_year}).to_netcdf('/g/data/k10/cb4968/data/cmip6/' + model + '/' + model + '_mse_' + experiment_id + '.nc')

NameError: name 'mse' is not defined

In [22]:
del ta, zg, hus

In [ ]:
mse_test = mse.isel(time=slice(0,4))

In [ ]:
mse_vInt = xr.DataArray(
    data=-scipy.integrate.simpson(mse.data, mse.plev.data, axis=1, even='last')/mse.plev.data[0],
    dims=['time','lat', 'lon'],
    coords={'time': mse.time.data, 'lat': mse.lat.data, 'lon': mse.lon.data}
    ,attrs={'units':''}
    )

In [ ]:
myPlots.plot_snapshot(mse_vInt.mean(dim=('time'), keep_attrs=True), 'viridis', 'time mean mse vInt', model)

In [ ]:
aWeights = np.cos(np.deg2rad(mse.lat))
f, ax = plt.subplots(figsize=(15, 4))

mse_vInt.weighted(aWeights).mean(dim=('lat','lon'), keep_attrs=True).plot(ax=ax,label='spatial mean mse')
ax.set_title('spatial mean, massweighted vertically integrated mse, model:' + model + ' exp:' + experiment_id)
plt.tight_layout()
plt.legend()

In [ ]:
aWeights = np.cos(np.deg2rad(mse.lat))
mse_tMean = mse_vInt.mean(dim=('time'), keep_attrs=True)
mse_sMean = mse_vInt.weighted(aWeights).mean(dim=('lat','lon'), keep_attrs=True)

In [ ]:
mseVint_anom = mse_vInt - mse_sMean

In [ ]:
mseVint_var = mseVint_anom**2

In [ ]:
dmse = mseVint_var.isel(time=slice(1,len(mse.time.data))).data - mseVint_var.isel(time=slice(0,len(mse.time.data)-1)).data

In [ ]:
dmse = xr.DataArray(
    data=dmse,
    dims=['time', 'lat', 'lon'],
    coords={'time': mse.time.data[0:len(mse.time.data)-1], 'lat': mse.lat.data, 'lon': mse.lon.data},
    attrs={'units':''}
    )

## saving

In [31]:
del zg_test.encoding['chunksizes']
#.to_netcdf(path, encoding=tas.encoding.update({'zlib': True, 'complevel': 4})

In [32]:
# mse test
save = True
if save:
    folder = '/g/data/k10/cb4968/cmip6/' + model
    fileName = model + '_mse_test_' + experiment_id + '.nc'
    dataSet = xr.Dataset({
                        'mse_test': mse_test, 
                        'ta_test': ta_test, 
                        'zg_test': zg_test,
                        'hus_test': hus_test})
    myFuncs.save_file(dataSet, folder, fileName)


In [ ]:
# mse tMean
save = True
if save:
    folder = '/g/data/k10/cb4968/cmip6/' + model
    fileName = model + '_mse_tMean_' + experiment_id + '.nc'
    dataSet = xr.Dataset({'mse_tMean': mse_tMean})
    myFuncs.save_file(dataSet, folder, fileName)


In [ ]:
# mse variance
save = True
if save:
    folder = '/g/data/k10/cb4968/cmip6/' + model
    fileName = model + '_mse_var_' + experiment_id + '.nc'
    dataSet = xr.Dataset({
                        'mse_var': mseVint_var.weighted(aWeights).mean(dim=('lat','lon'), keep_attrs=True),
                        'mse_anom': mseVint_anom.weighted(aWeights).mean(dim=('lat','lon'), keep_attrs=True),
                        'mse_sMean': mse_sMean})
    myFuncs.save_file(dataSet, folder, fileName)

In [ ]:
# mse dmse/dt
save = True
if save:
    folder = '/g/data/k10/cb4968/cmip6/' + model
    fileName = model + '_mse_dmse_' + experiment_id + '.nc'    
    xr.Dataset({'dmse ': dmse.weighted(aWeights).mean(dim=('lat','lon'), keep_attrs=True)})
    myFuncs.save_file(dataSet, folder, fileName)

## testing